<a href="https://colab.research.google.com/github/PreyPython123/Master-V24-Semiveiledet-Regresjon/blob/Collagen-Pradeep/Bioco_Collagen_Semiveiledet_Regresjonsmetoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importering, lasting og formattering

Lasting av nødvendige bibliotek og pakker

In [1]:
!pip install LAMDA-SSL
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.8/240.8 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 9.7 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


Importering av nødvendige bibliotek og pakker

In [3]:
import pandas as pd
import numpy as np
import copy
import math

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
import sklearn.model_selection
from statistics import mean

from LAMDA_SSL.Algorithm.Regression.CoReg import CoReg

import optuna
from optuna.visualization import plot_optimization_history

# Utelukker utskriving av logg for hyperoptimalisering
optuna.logging.set_verbosity(optuna.logging.WARNING)

Evalueringsmetrikker

In [4]:
from sklearn.metrics import (mean_absolute_error,
                             mean_squared_error,
                             mean_absolute_percentage_error,
                             r2_score)

# Lager en funksjon for RMSE
def rmse(y_faktisk, y_predikert):
  return np.sqrt(mean_squared_error(y_faktisk, y_predikert))

## Egendefinerte moduler

In [5]:
!pwd
! cp /content/drive/MyDrive/MasterV24/GitHub/oppdeling_trening_testsett.py /content/oppdeling_trening_testsett.py

/content


In [6]:
from oppdeling_trening_testsett import trening_testsett_oppdeling_enzym
from oppdeling_trening_testsett import trening_testsett_oppdeling_dag

In [7]:
def evaluering_beregning(y_test, y_test_prediksjon, y_trening, y_trening_prediksjon):
  rmse_test = rmse(y_test, y_test_prediksjon)
  r2_test = r2_score(y_test, y_test_prediksjon)
  mae_test = mean_absolute_error(y_test, y_test_prediksjon)
  mape_test = mean_absolute_percentage_error(y_test, y_test_prediksjon)

  rmse_trening = rmse(y_trening, y_trening_prediksjon)
  r2_trening = r2_score(y_trening, y_trening_prediksjon)
  mae_trening = mean_absolute_error(y_trening, y_trening_prediksjon)
  mape_trening = mean_absolute_percentage_error(y_trening, y_trening_prediksjon)

  return rmse_test, r2_test, mae_test, mape_test, rmse_trening, r2_trening, mae_trening, mape_trening

In [8]:
def evaluering_resultater(detailed_objective):
  rmse_test = detailed_objective(study.best_trial)[0]
  r2_test = detailed_objective(study.best_trial)[1]
  mae_test = detailed_objective(study.best_trial)[2]
  mape_test = detailed_objective(study.best_trial)[3]

  rmse_trening = detailed_objective(study.best_trial)[4]
  r2_trening = detailed_objective(study.best_trial)[5]
  mae_trening = detailed_objective(study.best_trial)[6]
  mape_trening = detailed_objective(study.best_trial)[7]

  return rmse_test, r2_test, mae_test, mape_test, rmse_trening, r2_trening, mae_trening, mape_trening

In [9]:
def rmse_validering_StratifiedKFold(modell, datasett, respons, n_splitt):

  skf = StratifiedKFold(n_splits=n_splitt)

  rmse_validering_resultat = []

  for trening, validering in skf.split(datasett, respons):
    X_trening_fold = (datasett.iloc[trening, :]).iloc[:, :-1]
    X_validering_fold = (datasett.iloc[validering, :]).iloc[:, :-1]

    y_trening_fold = (datasett.iloc[trening, :]).iloc[:, -1]
    y_validering_fold = (datasett.iloc[validering, :]).iloc[:, -1]

    modell.fit(X_trening_fold, y_trening_fold)

    y_validering_prediksjon = modell.predict(X_validering_fold)

    rmse_validering = rmse(y_validering_fold, y_validering_prediksjon)
    rmse_validering_resultat.append(rmse_validering)

  return mean(rmse_validering)

Importering av relevant data

In [10]:
# Velger første kolonne med dato og tid som index
collagen_data = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/collagen_data.csv',
                            header=0,
                            sep=',',
                            index_col=0)

# Normal produksjon
collagen_data_normal = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/collagen_data_normal.csv',
                                   header=0,
                                   sep=',',
                                   index_col=0)

# Importerer rådata
rå_data = pd.read_csv('/content/drive/MyDrive/MasterV24/BiocoData.csv',
                      header=0,
                      sep=';',
                      index_col=0)

# Formatterer index til riktig format og datatype
collagen_data.index = pd.to_datetime(collagen_data.index,
                                     format='%Y-%m-%d %H:%M:%S')

collagen_data_normal.index = pd.to_datetime(collagen_data_normal.index,
                                            format='%Y-%m-%d %H:%M:%S')

rå_data.index = pd.to_datetime(rå_data.index,
                               format='%d-%m-%Y %H:%M:%S.%f')

Tilfeldighetsfrø

In [11]:
#Tilfeldighetsfrø
random_seed = 123

Importering av relevant data

In [12]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [13]:
# Velger første kolonne med dato og tid som index
collagen_data = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/collagen_data.csv',
                            header=0,
                            sep=',',
                            index_col=0)

# Importerer rådata
rå_data = pd.read_csv('/content/drive/MyDrive/MasterV24/BiocoData.csv',
                      header=0,
                      sep=';',
                      index_col=0)

# Formatterer index til riktig format og datatype
collagen_data.index = pd.to_datetime(collagen_data.index,
                                     format='%Y-%m-%d %H:%M:%S')

rå_data.index = pd.to_datetime(rå_data.index,
                               format='%d-%m-%Y %H:%M:%S.%f')

Tilfeldighetsfrø

In [14]:
#Tilfeldighetsfrø
random_seed = 123

# NIR målinger inkludert

## Alternativ 1: Fordelt på enzymtyper

### Alternativ 1: Et testsett

In [19]:
# Velger testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
collagen_trening_markert, collagen_enzymtyper_markert,\
collagen_test, _, collagen_trening_umarkert,\
_, collagen_trening = trening_testsett_oppdeling_enzym(rå_data,
                                                       collagen_data,
                                                       test_andel=test_andel)

# Deler settene i forklarings- og responsvariabler
X_trening = collagen_trening.iloc[:, :-1]
X_trening_markert = collagen_trening_markert.iloc[:, :-1]
X_trening_umarkert = collagen_trening_umarkert.iloc[:, :-1]
X_test = collagen_test.iloc[:, :-1]

y_trening_markert = collagen_trening_markert.iloc[:, -1]
y_test = collagen_test.iloc[:, -1]

# Skalerer treningsdata
skalerer = StandardScaler()
skalerer.fit(X_trening)
X_trening_markert_transformert = skalerer.transform(X_trening_markert)
X_trening_umarkert_transformert = skalerer.transform(X_trening_umarkert)
X_test_transformert = skalerer.transform(X_test)

**CoReg**

Hyperparamter optimalisering med Optuna

In [25]:
def objective(trial):

  parametere = {
      'k1': trial.suggest_int('k1', 1, 10),
      'k2': trial.suggest_int('k2', 1, 10),
      'p1': trial.suggest_int('p1', 1, 10),
      'p2': trial.suggest_int('p2', 1, 10)
  }

  coreg = CoReg()

  splits = 2
  skf = StratifiedKFold(n_splits = splits)

  rmse_validering_resultat = []

  for train, test in skf.split(collagen_trening_markert, collagen_enzymtyper_markert):
    X_trening_fold = (collagen_trening_markert.iloc[train, :]).iloc[:, :-1]
    skalerer_fold = StandardScaler()
    skalerer_fold.fit(X_trening_fold)

    X_trening_fold_transformert = skalerer_fold.transform(X_trening_fold)
    X_validering_fold = (collagen_trening_markert.iloc[test, :]).iloc[:, :-1]
    X_validering_fold_transformert = skalerer_fold.transform(X_validering_fold)

    y_trening_fold = (collagen_trening_markert.iloc[train, :]).iloc[:, -1]
    y_validering_fold = (collagen_trening_markert.iloc[test, :]).iloc[:, -1]


    coreg.fit(X_trening_fold_transformert, y_trening_fold, X_trening_umarkert_transformert)
    y_validering_prediksjon = coreg.predict(X_validering_fold_transformert)
    rmse_validering = rmse(y_validering_fold, y_validering_prediksjon)
    rmse_validering_resultat.append(rmse_validering)

    rmse_validering_snitt = mean(rmse_validering_resultat)

    return rmse_validering_snitt

if __name__ == "__main__":
  study = optuna.create_study(direction='minimize')
  study.optimize(objective, n_trials=10)

plot_optimization_history(study)

Evauleringer av beste modell

In [26]:
def detailed_objective(trial):
  parametere = {
      'k1': trial.suggest_int('k1', 1, 10),
      'k2': trial.suggest_int('k2', 1, 10),
      'p1': trial.suggest_int('p1', 1, 10),
      'p2': trial.suggest_int('p2', 1, 10)
  }

  coreg = CoReg()

  coreg.fit(X_trening_markert_transformert, y_trening_markert.values, X_trening_umarkert_transformert)
  y_test_prediksjon = coreg.predict(X_test_transformert)
  y_trening_prediksjon = coreg.predict(X_trening_markert_transformert)

  rmse_test, r2_test,\
  mae_test, mape_test,\
  rmse_trening, r2_trening,\
  mae_trening, mape_trening = evaluering_beregning(y_test,
                                                    y_test_prediksjon,
                                                    y_trening_markert,
                                                    y_trening_prediksjon)

  return rmse_test, r2_test, mae_test, mape_test, rmse_trening, r2_trening, mae_trening, mape_trening

collagen_coreg_resultater_enzym = pd.DataFrame(columns = ["Test andel",
                                                          "RMSE test",
                                                          "R2 test",
                                                          "MAE test",
                                                          "MAPE test",
                                                          "RMSE trening",
                                                          "R2 trening",
                                                          "MAE trening",
                                                          "MAPE trening",
                                                          "Beste parametere"])

# Henter ut evalueringsresulateter av beste modell fra undersøkelse
rmse_test, r2_test, mae_test, mape_test,\
rmse_trening, r2_trening, mae_trening, mape_trening\
= evaluering_resultater(detailed_objective)

# Legger resulater til i datasett
beste_parametere = study.best_params

resultater = [test_andel,
              rmse_test, r2_test, mae_test, mape_test,
              rmse_trening, r2_trening, mae_trening, mape_trening,
              beste_parametere]

collagen_coreg_resultater_enzym.loc[len(collagen_coreg_resultater_enzym)] = resultater

In [27]:
collagen_coreg_resultater_enzym

,Test andel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening,Beste parametere
0,0.2,3.757312,0.601639,3.344107,0.149239,1.062904,0.946442,0.911116,0.039514,"{'k1': 7, 'k2': 8, 'p1': 8, 'p2': 8}"


### Alternativ 1: Flere testandeler

In [ ]:
collagen_coreg_resultater_enzym = pd.DataFrame(columns = ["Test andel",
                                                          "RMSE test",
                                                          "R2 test",
                                                          "MAE test",
                                                          "MAPE test",
                                                          "RMSE trening",
                                                          "R2 trening",
                                                          "MAE trening",
                                                          "MAPE trening",
                                                          "Beste parametere"])

test_andeler = [0.1, 0.2]

for test_andel in test_andeler:
  collagen_trening_markert, collagen_enzymtyper_markert,\
  collagen_test, _, collagen_trening_umarkert,\
  _, collagen_trening = trening_testsett_oppdeling_enzym(rå_data,
                                                        collagen_data,
                                                        test_andel=test_andel)

  X_trening = collagen_trening.iloc[:, :-1]
  X_trening_markert = collagen_trening_markert.iloc[:, :-1]
  X_trening_umarkert = collagen_trening_umarkert.iloc[:, :-1]
  X_test = collagen_test.iloc[:, :-1]

  y_trening_markert = collagen_trening_markert.iloc[:, -1]
  y_test = collagen_test.iloc[:, -1]

  skalerer = StandardScaler()
  skalerer.fit(X_trening)
  X_trening_markert_transformert = skalerer.transform(X_trening_markert)
  X_trening_umarkert_transformert = skalerer.transform(X_trening_umarkert)
  X_test_transformert = skalerer.transform(X_test)

  def objective(trial):
    parametere = {
      'k1': trial.suggest_int('k1', 1, 10),
      'k2': trial.suggest_int('k2', 1, 10),
      'p1': trial.suggest_int('p1', 1, 10),
      'p2': trial.suggest_int('p2', 1, 10)
  }

    coreg = CoReg()

    splits = 2
    skf = StratifiedKFold(n_splits = splits)

    rmse_validering_resultat = []

    for train, test in skf.split(collagen_trening_markert, collagen_enzymtyper_markert):
      X_trening_fold = (collagen_trening_markert.iloc[train, :]).iloc[:, :-1]
      skalerer_fold = StandardScaler()
      skalerer_fold.fit(X_trening_fold)

      X_trening_fold_transformert = skalerer_fold.transform(X_trening_fold)
      X_validering_fold = (collagen_trening_markert.iloc[test, :]).iloc[:, :-1]
      X_validering_fold_transformert = skalerer_fold.transform(X_validering_fold)

      y_trening_fold = (collagen_trening_markert.iloc[train, :]).iloc[:, -1]
      y_validering_fold = (collagen_trening_markert.iloc[test, :]).iloc[:, -1]


      coreg.fit(X_trening_fold_transformert, y_trening_fold, X_trening_umarkert_transformert)
      y_validering_prediksjon = coreg.predict(X_validering_fold_transformert)
      rmse_validering = rmse(y_validering_fold, y_validering_prediksjon)
      rmse_validering_resultat.append(rmse_validering)

      rmse_validering_snitt = mean(rmse_validering_resultat)

    return rmse_validering_snitt

  if __name__ == "__main__":
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=10)


  def detailed_objective(trial):
    parametere = {
        'k1': trial.suggest_int('k1', 1, 10),
        'k2': trial.suggest_int('k2', 1, 10),
        'p1': trial.suggest_int('p1', 1, 10),
        'p2': trial.suggest_int('p2', 1, 10)
    }

    coreg = CoReg()

    coreg.fit(X_trening_markert_transformert, y_trening_markert.values, X_trening_umarkert_transformert)
    y_test_prediksjon = coreg.predict(X_test_transformert)
    y_trening_prediksjon = coreg.predict(X_trening_markert_transformert)

    rmse_test, r2_test,\
    mae_test, mape_test,\
    rmse_trening, r2_trening,\
    mae_trening, mape_trening = evaluering_beregning(y_test,
                                                      y_test_prediksjon,
                                                      y_trening_markert,
                                                      y_trening_prediksjon)

    return rmse_test, r2_test, mae_test, mape_test, rmse_trening, r2_trening, mae_trening, mape_trening

  rmse_test, r2_test, mae_test, mape_test,\
  rmse_trening, r2_trening, mae_trening, mape_trening\
  = evaluering_resultater(detailed_objective)

  # Legger resulater til i datasett
  beste_parametere = study.best_params

  resultater = [test_andel,
                rmse_test, r2_test, mae_test, mape_test,
                rmse_trening, r2_trening, mae_trening, mape_trening,
                beste_parametere]

  collagen_coreg_resultater_enzym.loc[len(collagen_coreg_resultater_enzym)] = resultater

In [ ]:
collagen_coreg_resultater_enzym

,Test andel,RMSE test,R2 test,RMSE trening,R2 trening,Beste parametere
0,0.1,3.785645,0.545300,1.106640,0.943903,"{'k1': 2, 'k2': 10, 'p1': 2, 'p2': 2}"
1,0.2,3.782097,0.555504,1.214655,0.948109,"{'k1': 4, 'k2': 7, 'p1': 7, 'p2': 3}"
2,0.3,3.971840,0.594327,1.139087,0.949522,"{'k1': 2, 'k2': 5, 'p1': 3, 'p2': 2}"
3,0.4,3.923748,0.602700,1.219026,0.945218,"{'k1': 7, 'k2': 10, 'p1': 4, 'p2': 1}"
4,0.5,3.881073,0.617700,1.083216,0.949280,"{'k1': 9, 'k2': 10, 'p1': 9, 'p2': 10}"


## Alternativ 1: Flere treningsandeler

In [17]:
collagen_coreg_resulateter_enzym = pd.DataFrame(columns = ["Trenings andel",
                                                            "RMSE test",
                                                            "R2 test",
                                                            "MAE test",
                                                            "MAPE test",
                                                            "RMSE trening",
                                                            "R2 trening",
                                                            "MAE trening",
                                                            "MAPE trening",
                                                            "Beste parametere"])


# Setter fast testandel
test_andel = 0.1

# Deler datasettet i trening og testsett
collagen_trening_markert, collagen_enzymtyper_markert,\
collagen_test, _, collagen_trening_umarkert,\
_, collagen_trening = trening_testsett_oppdeling_enzym(rå_data,
                                                        collagen_data,
                                                        test_andel=test_andel)
# Treningssett
X_test = collagen_test.iloc[:, :-1]
y_test = collagen_test.iloc[:, -1]

In [18]:
# Andel av treningsettet som skal benyttes til trening av modell
trenings_andeler = [0.8, 0.9, 1.0]

for trenings_andel in trenings_andeler:
  print("Trening:", trenings_andel)
  if trenings_andel < 1.0:
      ubenyttet_andel = (1.0-trenings_andel)
      treningssett_markert, enzymtyper_trening_markert, _, _,\
      treningssett_umarkert, _, treningssett\
      =trening_testsett_oppdeling_enzym(rå_data,
                                        collagen_trening,
                                        test_andel=ubenyttet_andel,
                                        random_seed=random_seed)

  else:
    treningssett_markert = collagen_trening_markert
    enzymtyper_trening_markert = collagen_enzymtyper_markert
    treningssett_umarkert = collagen_trening_umarkert
    treningssett = collagen_trening


  X_trening = treningssett.iloc[:, :-1]
  X_trening_markert = treningssett_markert.iloc[:, :-1]
  X_trening_umarkert = treningssett_umarkert.iloc[:, :-1]
  y_trening_markert = treningssett_markert.iloc[:, -1]

  skalerer = StandardScaler()
  skalerer.fit(X_trening)
  X_trening_markert_transformert = skalerer.transform(X_trening_markert)
  X_trening_umarkert_transformert = skalerer.transform(X_trening_umarkert)
  X_test_transformert = skalerer.transform(X_test)

  def objective(trial):

    parametere = {
          'k1': trial.suggest_int('k1', 1, 5),
          'k2': trial.suggest_int('k2', 1, 5),
          'p1': trial.suggest_int('p1', 1, 5),
          'p2': trial.suggest_int('p2', 1, 5)
      }

    coreg = CoReg()

    splits = 2
    skf = StratifiedKFold(n_splits = splits)

    rmse_validering_resultat = []

    for train, test in skf.split(treningssett_markert, enzymtyper_trening_markert):
      print("Validering")
      X_trening_fold = (treningssett_markert.iloc[train, :]).iloc[:, :-1]
      skalerer_fold = StandardScaler()
      skalerer_fold.fit(X_trening_fold)

      X_trening_fold_transformert = skalerer_fold.transform(X_trening_fold)
      X_validering_fold = (treningssett_markert.iloc[test, :]).iloc[:, :-1]
      X_validering_fold_transformert = skalerer_fold.transform(X_validering_fold)

      y_trening_fold = (treningssett_markert.iloc[train, :]).iloc[:, -1]
      y_validering_fold = (treningssett_markert.iloc[test, :]).iloc[:, -1]


      coreg.fit(X_trening_fold_transformert, y_trening_fold, X_trening_umarkert_transformert)
      y_validering_prediksjon = coreg.predict(X_validering_fold_transformert)
      rmse_validering = rmse(y_validering_fold, y_validering_prediksjon)
      rmse_validering_resultat.append(rmse_validering)

    rmse_validering_snitt = mean(rmse_validering_resultat)

    return rmse_validering_snitt

  if __name__ == "__main__":
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=2)

  def detailed_objective(trial):
    parametere = {
        'k1': trial.suggest_int('k1', 1, 5),
        'k2': trial.suggest_int('k2', 1, 5),
        'p1': trial.suggest_int('p1', 1, 5),
        'p2': trial.suggest_int('p2', 1, 5)
    }

    coreg = CoReg()

    coreg.fit(X_trening_markert_transformert, y_trening_markert.values, X_trening_umarkert_transformert)
    y_test_prediksjon = coreg.predict(X_test_transformert)
    y_trening_prediksjon = coreg.predict(X_trening_markert_transformert)

    rmse_test, r2_test,\
    mae_test, mape_test,\
    rmse_trening, r2_trening,\
    mae_trening, mape_trening = evaluering_beregning(y_test,
                                                      y_test_prediksjon,
                                                      y_trening_markert,
                                                      y_trening_prediksjon)

    return rmse_test, r2_test, mae_test, mape_test, rmse_trening, r2_trening, mae_trening, mape_trening

  rmse_test, r2_test, mae_test, mape_test,\
  rmse_trening, r2_trening, mae_trening, mape_trening\
  = evaluering_resultater(detailed_objective)

  beste_parametere = study.best_params

  resultater = [test_andel,
                rmse_test, r2_test, mae_test, mape_test,
                rmse_trening, r2_trening, mae_trening, mape_trening,
                beste_parametere]

  collagen_coreg_resultater_enzym.loc[len(collagen_coreg_resultater_enzym)] = resultater

Trening: 0.8
Validering
Validering
Validering
Validering
Trening: 0.9
Validering
Validering


[W 2024-02-22 08:48:24,778] Trial 0 failed with parameters: {'k1': 1, 'k2': 1, 'p1': 4, 'p2': 5} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-18-a3f4eac5aeeb>", line 63, in objective
    coreg.fit(X_trening_fold_transformert, y_trening_fold, X_trening_umarkert_transformert)
  File "/usr/local/lib/python3.10/dist-packages/LAMDA_SSL/Algorithm/Regression/CoReg.py", line 79, in fit
    h_temp.predict(L_X[idx_o].reshape(1, -1))) ** 2
  File "/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_regression.py", line 236, in predict
    neigh_ind = self.kneighbors(X, return_distance=False)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_base.py", line 879, in kneighbors
    chunked_results = Parallel(n_jobs, prefer="threads")(
  File "/usr/local/lib/python3.10/dist-package

KeyboardInterrupt: 

## Alternativ 2: Fordelt på dag og kontinuitet

### Alternativ 2: Et testsett

In [ ]:
# Valg av testandel
test_andel = 0.2

# Oppdeling av datasett til trenin og testsett, samt markert og umarkert del
collagen_trening_markert, collagen_test, collagen_trening_umarkert, collagen_trening = trening_testsett_oppdeling_dag(rå_data,
                                                                                                                      collagen_data,
                                                                                                                      test_andel)
# Forklarings- og responsvariabler
X_trening = collagen_trening.iloc[:, :-1]
X_trening_markert = collagen_trening_markert.iloc[:, :-1]
X_trening_umarkert = collagen_trening_umarkert.iloc[:, :-1]
X_test = collagen_test.iloc[:, :-1]

y_trening_markert = collagen_trening_markert.iloc[:, -1]
y_test = collagen_test.iloc[:, -1]

# Skalering
skalerer = StandardScaler()
skalerer.fit(X_trening)
X_trening_markert_transformert = skalerer.transform(X_trening_markert)
X_trening_umarkert_transformert = skalerer.transform(X_trening_umarkert)
X_test_transformert = skalerer.transform(X_test)

<ipython-input-8-fc5ecd67e2cb>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_markert['RawMaterialMix'] = råmateriale_data


**CoReg**

Hyperparamter optimalisering med Optuna

In [ ]:
def objective(trial):
  parametere = {
      'k1': trial.suggest_int('k1', 1, 10),
      'k2': trial.suggest_int('k2', 1, 10),
      'p1': trial.suggest_int('p1', 1, 10),
      'p2': trial.suggest_int('p2', 1, 10)
  }

  coreg = CoReg()

  coreg.fit(X_trening_markert_transformert, y_trening_markert.values, X_trening_umarkert_transformert)
  y_test_prediksjon = coreg.predict(X_test_transformert)
  rmse_test = rmse(y_test, y_test_prediksjon)
  return rmse_test

if __name__ == "__main__":
  study = optuna.create_study(direction='minimize')
  study.optimize(objective, n_trials=10)

plot_optimization_history(study)

[I 2024-02-15 14:28:53,433] A new study created in memory with name: no-name-4a0c3411-e2ea-4af7-95d2-e672050f4c97
[I 2024-02-15 14:30:28,930] Trial 0 finished with value: 3.946188071342318 and parameters: {'k1': 9, 'k2': 3, 'p1': 9, 'p2': 10}. Best is trial 0 with value: 3.946188071342318.
[I 2024-02-15 14:32:09,881] Trial 1 finished with value: 3.8542007899294615 and parameters: {'k1': 4, 'k2': 1, 'p1': 1, 'p2': 1}. Best is trial 1 with value: 3.8542007899294615.
[I 2024-02-15 14:33:46,219] Trial 2 finished with value: 3.7641835996585167 and parameters: {'k1': 8, 'k2': 3, 'p1': 3, 'p2': 10}. Best is trial 2 with value: 3.7641835996585167.
[I 2024-02-15 14:35:20,185] Trial 3 finished with value: 3.670252371335196 and parameters: {'k1': 9, 'k2': 8, 'p1': 6, 'p2': 2}. Best is trial 3 with value: 3.670252371335196.
[I 2024-02-15 14:36:56,081] Trial 4 finished with value: 3.6237016038207743 and parameters: {'k1': 6, 'k2': 2, 'p1': 2, 'p2': 9}. Best is trial 4 with value: 3.6237016038207743

Evauleringer av beste modell

In [ ]:
def detailed_objective(trial):
  parametere = {
      'k1': trial.suggest_int('k1', 1, 10),
      'k2': trial.suggest_int('k2', 1, 10),
      'p1': trial.suggest_int('p1', 1, 10),
      'p2': trial.suggest_int('p2', 1, 10)
  }

  coreg = CoReg()

  coreg.fit(X_trening_markert_transformert, y_trening_markert.values, X_trening_umarkert_transformert)
  y_test_prediksjon = coreg.predict(X_test_transformert)
  y_trening_prediksjon = coreg.predict(X_trening_markert_transformert)

  rmse_test = rmse(y_test, y_test_prediksjon)
  r2_test = r2_score(y_test, y_test_prediksjon)
  mae_test = mean_absolute_error(y_test, y_test_prediksjon)
  mape_test = mean_absolute_percentage_error(y_test, y_test_prediksjon)


  rmse_trening = rmse(y_trening_markert, y_trening_prediksjon)
  r2_trening = r2_score(y_trening_markert, y_trening_prediksjon)
  mae_trening = mean_absolute_error(y_trening_markert, y_trening_prediksjon)
  mape_trening = mean_absolute_percentage_error(y_trening_markert, y_trening_prediksjon)

  return rmse_test, r2_test, mae_test, mape_test, rmse_trening, r2_trening, mae_trening, mape_trening

rmse_test_resultat = detailed_objective(study.best_trial)[0]
r2_test_resultat = detailed_objective(study.best_trial)[1]
mae_test_resultat = detailed_objective(study.best_trial)[2]
mape_test_resultat = detailed_objective(study.best_trial)[3]

rmse_trening_resultat = detailed_objective(study.best_trial)[4]
r2_trening_resultat = detailed_objective(study.best_trial)[5]
mae_trening_resultat = detailed_objective(study.best_trial)[6]
mape_trening_resultat = detailed_objective(study.best_trial)[7]

collagen_coreg_resultater_enzym = pd.DataFrame(columns = ["Test andel",
                                                          "RMSE test",
                                                          "R2 test",
                                                          "MAE test",
                                                          "MAPE test",
                                                          "RMSE trening",
                                                          "R2 trening",
                                                          "MAE trening",
                                                          "MAPE trening",
                                                          "Beste parametere"])

beste_parametere = study.best_params

resultater = [test_andel,
              rmse_test_resultat,
              r2_test_resultat,
              mae_test_resultat,
              mape_test_resultat,
              rmse_trening_resultat,
              r2_trening_resultat,
              mae_trening_resultat,
              mape_trening_resultat,
              beste_parametere]

collagen_coreg_resultater_enzym.loc[len(collagen_coreg_resultater_enzym)] = resultater
print(collagen_coreg_resultater_enzym)

   Test andel  RMSE test   R2 test  MAE test  MAPE test  RMSE trening  \
0         0.2   3.654918  0.453441  3.058378   0.146384      1.390003   

   R2 trening  MAE trening  MAPE trening                      Beste parametere  
0    0.925645     0.997047      0.044034  {'k1': 8, 'k2': 2, 'p1': 3, 'p2': 7}  


### Alternativ 2: Flere testandeler

In [ ]:
collagen_coreg_resultater_dag = pd.DataFrame(columns = ["Test andel",
                                                        "RMSE test",
                                                        "R2 test",
                                                        "MAE test",
                                                        "MAPE test",
                                                        "RMSE trening",
                                                        "R2 trening",
                                                        "MAE trening",
                                                        "MAPE trening",
                                                        "Beste parametere"])

test_andeler = [0.1, 0.2, 0.3, 0.4, 0.5]

for test_andel in test_andeler:
  collagen_trening_markert, collagen_test, collagen_trening_umarkert, collagen_trening = trening_testsett_oppdeling_dag(rå_data,
                                                                                                                      collagen_data,
                                                                                                                      test_andel)


  X_trening = collagen_trening.iloc[:, :-1]
  X_trening_markert = collagen_trening_markert.iloc[:, :-1]
  X_trening_umarkert = collagen_trening_umarkert.iloc[:, :-1]
  X_test = collagen_test.iloc[:, :-1]

  y_trening_markert = collagen_trening_markert.iloc[:, -1]
  y_test = collagen_test.iloc[:, -1]

  skalerer = StandardScaler()
  skalerer.fit(X_trening)
  X_trening_markert_transformert = skalerer.transform(X_trening_markert)
  X_trening_umarkert_transformert = skalerer.transform(X_trening_umarkert)
  X_test_transformert = skalerer.transform(X_test)

  def objective(trial):
    parametere = {
        'k1': trial.suggest_int('k1', 1, 10),
        'k2': trial.suggest_int('k2', 1, 10),
        'p1': trial.suggest_int('p1', 1, 10),
        'p2': trial.suggest_int('p2', 1, 10)
    }

    coreg = CoReg()

    coreg.fit(X_trening_markert_transformert, y_trening_markert.values, X_trening_umarkert_transformert)
    y_test_prediksjon = coreg.predict(X_test_transformert)
    rmse_test = rmse(y_test, y_test_prediksjon)
    return rmse_test

  if __name__ == "__main__":
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=10)


  def detailed_objective(trial):
    parametere = {
        'k1': trial.suggest_int('k1', 1, 10),
        'k2': trial.suggest_int('k2', 1, 10),
        'p1': trial.suggest_int('p1', 1, 10),
        'p2': trial.suggest_int('p2', 1, 10)
    }

    coreg = CoReg()

    coreg.fit(X_trening_markert_transformert, y_trening_markert.values, X_trening_umarkert_transformert)
    y_test_prediksjon = coreg.predict(X_test_transformert)
    y_trening_prediksjon = coreg.predict(X_trening_markert_transformert)

    rmse_test = rmse(y_test, y_test_prediksjon)
    r2_test = r2_score(y_test, y_test_prediksjon)
    mae_test = mean_absolute_error(y_test, y_test_prediksjon)
    mape_test = mean_absolute_percentage_error(y_test, y_test_prediksjon)


    rmse_trening = rmse(y_trening_markert, y_trening_prediksjon)
    r2_trening = r2_score(y_trening_markert, y_trening_prediksjon)
    mae_trening = mean_absolute_error(y_trening_markert, y_trening_prediksjon)
    mape_trening = mean_absolute_percentage_error(y_trening_markert, y_trening_prediksjon)

    return rmse_test, r2_test, mae_test, mape_test, rmse_trening, r2_trening, mae_trening, mape_trening

  rmse_test_resultat = detailed_objective(study.best_trial)[0]
  r2_test_resultat = detailed_objective(study.best_trial)[1]
  mae_test_resultat = detailed_objective(study.best_trial)[2]
  mape_test_resultat = detailed_objective(study.best_trial)[3]

  rmse_trening_resultat = detailed_objective(study.best_trial)[4]
  r2_trening_resultat = detailed_objective(study.best_trial)[5]
  mae_trening_resultat = detailed_objective(study.best_trial)[6]
  mape_trening_resultat = detailed_objective(study.best_trial)[7]

  beste_parametere = study.best_params

  resultater = [test_andel,
                rmse_test_resultat,
                r2_test_resultat,
                mae_test_resultat,
                mape_test_resultat,
                rmse_trening_resultat,
                r2_trening_resultat,
                mae_trening_resultat,
                mape_trening_resultat,
                beste_parametere]

  collagen_coreg_resultater_dag.loc[len(collagen_coreg_resultater_dag)] = resultater

<ipython-input-8-fc5ecd67e2cb>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_markert['RawMaterialMix'] = råmateriale_data
[I 2024-02-16 12:02:10,534] A new study created in memory with name: no-name-d74b9daf-55c3-44f0-b2a5-598063eac103
[I 2024-02-16 12:03:36,415] Trial 0 finished with value: 3.8877537424688855 and parameters: {'k1': 3, 'k2': 8, 'p1': 1, 'p2': 9}. Best is trial 0 with value: 3.8877537424688855.
[I 2024-02-16 12:05:00,324] Trial 1 finished with value: 4.3391025260368075 and parameters: {'k1': 8, 'k2': 10, 'p1': 2, 'p2': 8}. Best is trial 0 with value: 3.8877537424688855.
[I 2024-02-16 12:06:24,139] Trial 2 finished with value: 3.287827978347458 and parameters: {'k1': 7, 'k2': 9, 'p1': 9, 'p2': 3}. Best is trial 2 with valu

In [ ]:
collagen_coreg_resultater_dag

,Test andel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening,Beste parametere
0,0.1,3.732778,0.105147,3.265388,0.162898,1.462619,0.927568,1.056689,0.046517,"{'k1': 7, 'k2': 9, 'p1': 9, 'p2': 3}"
1,0.2,3.516569,0.417800,3.157804,0.147428,1.351680,0.926205,1.045626,0.046103,"{'k1': 6, 'k2': 5, 'p1': 6, 'p2': 1}"
2,0.3,4.132328,0.045528,3.398061,0.168199,1.339703,0.932920,1.020832,0.044069,"{'k1': 9, 'k2': 6, 'p1': 6, 'p2': 1}"
3,0.4,3.490240,0.540573,2.805247,0.124713,1.446574,0.931633,1.177557,0.053503,"{'k1': 7, 'k2': 8, 'p1': 2, 'p2': 8}"
4,0.5,3.917921,0.475110,2.899328,0.148522,1.398529,0.908545,1.139016,0.045939,"{'k1': 3, 'k2': 9, 'p1': 5, 'p2': 10}"
